In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RandomizedSearchCV
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer
from sklearn.neural_network import MLPRegressor
import time

In [ ]:
df = pd.read_excel('Data diolah darin.xlsx')
df

,Backhoe,HP(watt),Norm_MAP,RPM,Age,Load_Factor,Engine_Tier,TEMP[C],Fuel[g/s],NOx[g/s],...,NOx[g/hr],HC[g/hr],CO[g/hr],CO2[g/hr],PM[mg/hr],Nox (g/kl),HC (g/kl),CO (g/kl),CO2 (g/kl),PM (g/kl)
0,1,65621.6,0.000000,833,12,0.21,2,22,0.430000,0.024090,...,86.724000,25.848000,13.176000,4780.656000,14.4,3730.117500,1111.757728,566.717728,2.056225e+05,0.619362
1,1,65621.6,0.012346,800,12,0.21,2,22,0.620000,0.030320,...,109.152000,25.668000,16.164000,7008.264000,21.6,4694.776362,1104.015681,695.235681,3.014350e+05,0.929047
2,1,65621.6,0.012346,826,12,0.21,2,22,0.510000,0.030730,...,110.628000,10.980000,10.584000,5722.920000,21.6,4758.261138,472.264772,455.232272,2.461506e+05,0.929047
3,1,65621.6,0.012346,831,12,0.21,2,22,0.390522,0.023693,...,85.293664,20.747853,4.825082,4382.553038,21.6,3668.596780,892.393432,207.533355,1.884996e+05,0.929047
4,1,65621.6,0.012346,834,12,0.21,2,22,0.413024,0.026107,...,93.985262,6.979907,9.880556,4673.503069,21.6,4042.434273,300.215326,424.976193,2.010137e+05,0.929047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37523,8,72332.9,0.975904,1676,36,0.21,2,41,1.590000,0.058830,...,211.788000,19.908000,28.152000,17957.592000,1800.0,445343.100000,41862.100000,59197.400000,3.776083e+07,3785.000000
37524,8,72332.9,0.975904,1683,36,0.21,2,42,1.320000,0.059500,...,214.200000,19.800000,30.780000,14905.044000,1656.0,542545.340900,50151.250000,77962.397730,3.775286e+07,4194.468182
37525,8,72332.9,0.975904,1846,36,0.21,2,42,1.530000,0.059640,...,214.704000,21.744000,33.840000,17349.048000,1872.0,469179.694100,47515.850980,73948.509800,3.791183e+07,4090.768627
37526,8,72332.9,0.987952,1876,36,0.21,2,42,1.760000,0.056200,...,202.320000,22.356000,34.740000,19948.572000,1980.0,384340.943200,42468.990340,65994.485800,3.789568e+07,3761.343750


In [ ]:
df.replace('#DIV/0!',np.nan,inplace=True)
df.dropna(inplace=True)

C:\Users\TI 2023\AppData\Local\Temp\ipykernel_6200\1019318869.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace('#DIV/0!',np.nan,inplace=True)


In [ ]:
X = df[['Backhoe', 'HP(watt)', 'Norm_MAP', 'RPM', 'Age', 'Load_Factor', 'Engine_Tier']]
y = df['NOx[g/s]']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

y_train = y_train.ravel()
y_test = y_test.ravel()

C:\Users\TI 2023\AppData\Local\Temp\ipykernel_6200\1236518287.py:3: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  y_train = y_train.ravel()
C:\Users\TI 2023\AppData\Local\Temp\ipykernel_6200\1236518287.py:4: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  y_test = y_test.ravel()


In [ ]:
mlp = MLPRegressor()
mlp

MLPRegressor()

In [ ]:
start_time = time.time()
mlp.fit(X_train, y_train)
print("Execution time: " + str((time.time() - start_time)) + ' s')

Execution time: 1.3489348888397217 s


In [ ]:
y_pred = mlp.predict(X_test)

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE: {rmse:.2f}")

RMSE: 1.30


In [ ]:
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f"MAPE: {mape:.2f}%")

MAPE: 66.22%


In [ ]:
param = {
    'hidden_layer_sizes': range(1, 100),
    'activation':['relu','tanh', 'logistic', 'identity'],
    'alpha': np.arange(0.01, 1.0, 0.01),
    'learning_rate':['invscaling','adaptive'],
}

rmse_scorer = make_scorer(mean_squared_error, squared=False)
mape_scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)


Random

In [ ]:
rs = RandomizedSearchCV(mlp,
                            param,
                            cv=3,
                            scoring={'RMSE': rmse_scorer, 'MAPE': mape_scorer},
                            refit='RMSE',
                            n_iter=100,
                            verbose=1,
                            n_jobs=-1
                            )


start_time = time.time()
rs.fit(X_train, y_train)


print("Best: %f using %s" % (rs.best_score_, rs.best_params_))
print("Best RMSE:", rs.best_score_)
print("Best MAPE:", rs.cv_results_['mean_test_MAPE'][rs.best_index_])
print("Execution time: " + str((time.time() - start_time)) + ' s')

Fitting 3 folds for each of 100 candidates, totalling 300 fits
Best: 50.014029 using {'learning_rate': 'adaptive', 'hidden_layer_sizes': 89, 'alpha': 0.47000000000000003, 'activation': 'identity'}
Best RMSE: 50.014028935296814
Best MAPE: -2640.8360398253003
Execution time: 26.6285297870636 s


In [ ]:
best_rs = rs.best_estimator_
y_pred_rs = best_rs.predict(X_test)
rmse_rs = np.sqrt(mean_squared_error(y_test, y_pred_rs))
mape_rs = mean_absolute_percentage_error(y_test, y_pred_rs)
print(f"RMSE: {rmse_rs}")
print(f"MAPE: {mape_rs:.2f}%")

RMSE: 12.465156004235945
MAPE: 666.42%


In [ ]:
pd.DataFrame(rs.cv_results_).to_excel('MLP_RS_unscal.xlsx', index=False)

Bayessian

In [ ]:
def mlp_bs(hidden_layer_sizes, activation, alpha, learning_rate):
    activations = ['relu','tanh', 'logistic', 'identity']
    learning_rates = ['invscaling','adaptive']
    mlpbs= MLPRegressor(
        hidden_layer_sizes=int(hidden_layer_sizes),
        activation=activations[int(activation)],
        alpha=float(alpha),
        learning_rate=learning_rates[int(learning_rate)],
    )
    scores = cross_val_score(mlpbs, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    rmse = np.sqrt(-scores.mean())
    return -rmse

In [ ]:
bayes_opt = BayesianOptimization(
    mlp_bs,
    {
        'hidden_layer_sizes': (1, 100),
        'activation': (0, 3),
        'alpha': (0.01, 1.0),
        'learning_rate': (0, 1)
    }
)

In [ ]:
start_time = time.time()
bayes_opt.maximize(init_points=10, n_iter=100)
print("Execution time: " + str((time.time() - start_time)) + ' s')

|   iter    |  target   | activa... |   alpha   | hidden... | learni... |
-------------------------------------------------------------------------
| 1         | -0.1036   | 1.391     | 0.01022   | 96.4      | 0.7494    |
| 2         | -0.06312  | 1.986     | 0.222     | 51.42     | 0.2252    |
| 3         | -0.5175   | 1.152     | 0.7258    | 7.52      | 0.4037    |
| 4         | -0.01937  | 2.25      | 0.589     | 10.42     | 0.5575    |
| 5         | -0.02355  | 2.588     | 0.1518    | 45.38     | 0.9565    |
| 6         | -0.01939  | 1.791     | 0.1272    | 69.02     | 0.8764    |
| 7         | -0.01943  | 2.917     | 0.955     | 37.11     | 0.5417    |
| 8         | -0.01943  | 2.792     | 0.6176    | 46.77     | 0.8147    |
| 9         | -1.527    | 0.4652    | 0.1039    | 81.95     | 0.3661    |
| 10        | -0.0193   | 1.895     | 0.6294    | 42.27     | 0.1083    |
| 11        | -24.23    | 3.0       | 0.259     | 18.67     | 0.9395    |
| 12        | -1.219    | 0.0       | 

c:\Users\TI 2023\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


| 17        | -0.9072   | 3.0       | 1.0       | 1.0       | 1.0       |
| 18        | -24.81    | 3.0       | 0.01      | 56.73     | 0.0       |
| 19        | -21.29    | 3.0       | 1.0       | 64.47     | 0.0       |
| 20        | -21.75    | 3.0       | 1.0       | 73.21     | 1.0       |


c:\Users\TI 2023\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


| 21        | -1.824    | 0.0       | 0.01      | 3.055     | 0.0       |
| 22        | -17.56    | 3.0       | 1.0       | 79.11     | 0.0       |
| 23        | -1.067    | 0.0       | 0.01      | 48.08     | 0.0       |
| 24        | -2.336    | 0.0       | 0.01      | 38.77     | 1.0       |
| 25        | -15.81    | 3.0       | 1.0       | 25.76     | 1.0       |
| 26        | -2.028    | 0.0       | 0.01      | 13.52     | 0.0       |
| 27        | -4.598    | 3.0       | 0.01      | 4.836     | 1.0       |
| 28        | -4.713    | 0.0       | 0.01      | 66.5      | 1.0       |
| 29        | -1.8      | 0.0       | 1.0       | 70.75     | 0.0       |
| 30        | -1.943    | 0.0       | 1.0       | 53.38     | 1.0       |
| 31        | -20.17    | 3.0       | 1.0       | 98.59     | 0.0       |
| 32        | -40.82    | 3.0       | 1.0       | 33.32     | 1.0       |
| 33        | -16.12    | 3.0       | 0.01      | 39.61     | 0.0       |
| 34        | -5.336    | 0.0       | 

c:\Users\TI 2023\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\TI 2023\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\TI 2023\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


| 41        | -0.0379   | 0.0       | 1.0       | 1.0       | 1.0       |
| 42        | -1.17     | 0.0       | 0.01      | 26.49     | 0.0       |
| 43        | -57.66    | 3.0       | 1.0       | 49.43     | 0.0       |
| 44        | -2.452    | 0.5301    | 0.147     | 51.89     | 0.6527    |
| 45        | -0.01933  | 1.623     | 1.0       | 2.698     | 1.0       |
| 46        | -2.888    | 0.08735   | 1.0       | 5.242     | 1.0       |
| 47        | -0.01931  | 2.57      | 0.01      | 53.18     | 1.0       |
| 48        | -1.169    | 0.0       | 1.0       | 94.71     | 1.0       |
| 49        | -13.93    | 3.0       | 0.01      | 8.204     | 1.0       |
| 50        | -2.056    | 0.0       | 1.0       | 41.09     | 0.0       |
| 51        | -0.4437   | 0.0       | 1.0       | 84.28     | 1.0       |
| 52        | -0.01947  | 2.717     | 1.0       | 83.3      | 1.0       |
| 53        | -13.42    | 3.0       | 0.01      | 94.72     | 1.0       |
| 54        | -22.02    | 0.4831    | 

In [ ]:
best_iter = np.argmin([res['target'] for res in bayes_opt.res])

In [ ]:
best_params = bayes_opt.res[best_iter]['params']

print("Best Parameters:", best_params)

Best Parameters: {'activation': 3.0, 'alpha': 1.0, 'hidden_layer_sizes': 89.38938791490266, 'learning_rate': 0.0}


In [ ]:
param = {
    'hidden_layer_sizes': range(1, 100),
    'activation':['relu','tanh', 'logistic', 'identity'],
    'alpha': np.arange(0.01, 1.0, 0.01),
    'learning_rate':['invscaling','adaptive'],
}

rmse_scorer = make_scorer(mean_squared_error, squared=False)
mape_scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)

In [ ]:
activations = ['relu','tanh', 'logistic', 'identity']
learning_rates = ['invscaling','adaptive']

best_bs = MLPRegressor(
    hidden_layer_sizes=int(best_params['hidden_layer_sizes']),
    activation=activations[int(best_params['activation'])],  # Convert int to string
    alpha=float(best_params['alpha']),
    learning_rate=learning_rates[int(best_params['learning_rate'])]  # Convert int to string
)
best_bs.fit(X_train, y_train)

MLPRegressor(activation='identity', alpha=1.0, hidden_layer_sizes=89,
             learning_rate='invscaling')

In [ ]:
y_pred_bs = best_bs.predict(X_test)
rmse_bs = np.sqrt(mean_squared_error(y_test, y_pred_bs))
mape_bs = mean_absolute_percentage_error(y_test, y_pred_bs)

print(f"RMSE: {rmse_bs}")
print(f"MAPE: {mape_bs:.2f}%")

RMSE: 13.891566634084327
MAPE: 715.74%
